In [1]:
from utils_filter import *
from utils_datetime import *
from utils_geography import *
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors

In [2]:
# read in data
data_location = 'data'
moderate = False # only consider moderate days
labelled = True # start with already labelled data
outlooks, pph, reports = read_datasets(data_location, moderate, labelled)

reading outlooks
reading pph
reading storm reports


In [15]:
def plot_ramps(outlooks, title_insert=''):
    # 2d histogram of ramps
    # Bar chart of region, season, max category, 4 ramp. Same for just moderate and up
    ramp_up_bins = [-.5, .5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5]
    ramp_up_amounts = [0, 1, 2, 3, 4, 5, 6]
    ramp_down_amounts = [-6, -5, -4, -3, -2, -1, 0]
    ramp_down_bins = [-6.5, -5.5, -4.5, -3.5, -2.5, -1.5, -.5, .5]
    heatmap = np.histogram2d(np.array(outlooks['RAMP_UP'], dtype=int), np.array(outlooks['RAMP_DOWN'], dtype=int), bins = (ramp_up_bins, ramp_down_bins))
    im = plt.imshow(heatmap[0],  norm=colors.LogNorm())
    plt.colorbar(im)
    bins = len(ramp_up_bins)
    for i in range(bins-1): 
        for j in range(bins-1): 
            plt.annotate(str(int(heatmap[0][i][j])), xy=(j, i), 
                        ha='center', va='center', color='black') 

    plt.xticks(ramp_up_amounts, labels=ramp_down_amounts)
    plt.yticks(ramp_up_amounts, labels=ramp_up_amounts)
    plt.xlabel("Ramp Down")
    plt.ylabel("Ramp Up")
    plt.title("Number of " + title_insert + "Days with each Ramp Up and Ramp Down Amount")
    plt.show()

In [19]:
outlooks['DATE'].unique()

array(['198701090000', '198701160000', '198701190000', ...,
       '202312310000', '202401010000', '202401020000'], dtype=object)

In [12]:
pph['RAMP_UP'].values

array(['NONE', 'NONE', 'NONE', ..., '0', '1', '0'], dtype=object)